In [1]:
import pandas as pd

In [61]:
# Parse the raw text into a dataframe

filename = 'Collocations'

with open(filename, 'r') as f:
    raw_text = f.read()
    
text_list = raw_text.strip().split(' ')

ngram = (
    pd.DataFrame(data=text_list, columns=['unigram'])
    .assign(unigram = lambda x: x['unigram'].str.strip())
    .loc[lambda x: ~x['unigram'].isin([',', '.', "'", '"', '`', '-', ':'])]
    .assign(second_word = lambda x: x['unigram'].shift(-1).fillna('EndOfDocument'))
    .assign(bigram = lambda x: x['unigram'].str.cat(x['second_word'], sep='::'))
    .drop(['second_word'], axis=1)
)

In [63]:
ngram.tail()

,unigram,bigram
484375,N.V.,N.V.::the
484377,the,the::Dutch
484378,Dutch,Dutch::publishing
484379,publishing,publishing::group
484380,group,group::EndOfDocument


In [99]:
unigram_count = (
    ngram
    .groupby(by=['unigram'])
    .agg({'unigram': 'count'})
    .rename(columns={'unigram': 'count'})
    .sort_values(by='count', ascending=False)
    ['count']
)

bigram_count = (
    ngram.groupby(by=['bigram'])
    .agg({'bigram': 'count'})
    .rename(columns={'bigram': 'count'})
    .sort_values(by='count', ascending=False)
    ['count']
)